In [1]:
## 4
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Merge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return numpy.array(dataX), numpy.array(dataY)

corMat = pandas.read_csv('correlationmatrix.csv', engine='python')
corMat = corMat.iloc[:,1:21]

dataframe = pandas.read_csv('시뮬레이션 데이터.csv', engine='python')
dataframe = dataframe.iloc[0:46,1:21]
error_scores2= pandas.DataFrame(index=range(20),columns=["5~10년","5~15년","5~20년","5~25년","5~30년","5~35년"])

for i in range(20):
	k=0
	dataframe2 = dataframe.iloc[:,i]
	dataset = dataframe2.values
	dataset = dataset.astype('float32')
	scaler = MinMaxScaler(feature_range=(0, 1))
	dataset = scaler.fit_transform(dataset)
	
	datasetcompare1 = (dataframe.iloc[:,corMat.iloc[i,1]]).values
	datasetcompare1 = datasetcompare1.astype('float32')
	scalercompare1 = MinMaxScaler(feature_range=(0, 1))
	datasetcompare1 = scalercompare1.fit_transform(datasetcompare1)
	

	datasetcompare2 = (dataframe.iloc[:,corMat.iloc[i,2]]).values
	datasetcompare2 = datasetcompare2.astype('float32')
	scalercompare2 = MinMaxScaler(feature_range=(0, 1))
	datasetcompare2 = scalercompare2.fit_transform(datasetcompare2)

	datasetcompare3 = (dataframe.iloc[:,corMat.iloc[i,3]]).values
	datasetcompare3 = datasetcompare3.astype('float32')
	scalercompare3 = MinMaxScaler(feature_range=(0, 1))
	datasetcompare3 = scalercompare3.fit_transform(datasetcompare3)
	
	repeats = 3
	
	for j in [5,10,15,20,25,30]:
		error_scores = list()
		for r in range(repeats):
			numpy.random.seed(random.randrange(1,1000))
			train_size = j
			test_size = len(dataset) - train_size
			train, test = dataset[0:train_size], dataset[train_size:len(dataset)]
			look_back = 1
			trainX, trainY = create_dataset(train, look_back)
			testX, testY = create_dataset(test, look_back)
			trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
			testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
			batch_size = 1
			first_model = Sequential()
			first_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			first_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			first_model.add(Dense(1))
			
			
			traincompare1, testcompare1 = datasetcompare1[0:train_size], datasetcompare1[train_size:len(dataset)]
			look_back = 1
			trainXcompare1, trainYcompare1 = create_dataset(traincompare1, look_back)
			testXcompare1, testYcompare1 = create_dataset(testcompare1, look_back)
			trainXcompare1 = numpy.reshape(trainXcompare1, (trainXcompare1.shape[0], 1, trainXcompare1.shape[1]))
			testXcompare1 = numpy.reshape(testXcompare1, (testXcompare1.shape[0], 1, testXcompare1.shape[1]))
			
			second_model = Sequential()
			second_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			second_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			second_model.add(Dense(1))
			
			
			traincompare2, testcompare2 = datasetcompare2[0:train_size], datasetcompare2[train_size:len(dataset)]
			look_back = 1
			trainXcompare2, trainYcompare2 = create_dataset(traincompare2, look_back)
			testXcompare2, testYcompare2 = create_dataset(testcompare2, look_back)
			trainXcompare2 = numpy.reshape(trainXcompare2, (trainXcompare2.shape[0], 1, trainXcompare2.shape[1]))
			testXcompare2 = numpy.reshape(testXcompare2, (testXcompare2.shape[0], 1, testXcompare2.shape[1]))
			
			third_model = Sequential()
			third_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			third_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			third_model.add(Dense(1))

			
			traincompare3, testcompare3 = datasetcompare3[0:train_size], datasetcompare3[train_size:len(dataset)]
			look_back = 1
			trainXcompare3, trainYcompare3 = create_dataset(traincompare3, look_back)
			testXcompare3, testYcompare3 = create_dataset(testcompare3, look_back)
			trainXcompare3 = numpy.reshape(trainXcompare3, (trainXcompare3.shape[0], 1, trainXcompare3.shape[1]))
			testXcompare3 = numpy.reshape(testXcompare3, (testXcompare3.shape[0], 1, testXcompare3.shape[1]))
			
            
			fourth_model = Sequential()
			fourth_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			fourth_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			fourth_model.add(Dense(1))
			
			
			
			
			model = Sequential()
			model.add(Merge([first_model, second_model,third_model,fourth_model], mode='concat'))
			model.add(Dense(1))

			model.compile(loss='mean_squared_error', optimizer='adam')
			for z in range(10):           
				model.fit([trainX,trainXcompare1,trainXcompare2,trainXcompare3], trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
				model.reset_states()
			trainPredict = model.predict([trainX,trainXcompare1,trainXcompare2,trainXcompare3], batch_size=batch_size)
			model.reset_states()
			testPredict = model.predict([testX,testXcompare1,testXcompare2,testXcompare3], batch_size=batch_size)
			trainPredict = scaler.inverse_transform(trainPredict)
			trainY = scaler.inverse_transform([trainY])
			testPredict = scaler.inverse_transform(testPredict)
			testY = scaler.inverse_transform([testY])
			
			
			testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
			error_scores.append(testScore)
		
		results = pandas.DataFrame()
		results['rmse'] = error_scores
		error_scores2.iloc[i,k]=results.describe().iloc[1,0]
		k=k+1
		

Using TensorFlow backend.
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.

Epoch 1/1


InternalError: Blas GEMM launch failed : a.shape=(1, 4), b.shape=(4, 4), m=1, n=4, k=4
	 [[Node: lstm_3/while/MatMul_4 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](lstm_3/while/Identity_2, lstm_3/while/MatMul_4/Enter)]]
	 [[Node: loss/mul/_297 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_11150_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'lstm_3/while/MatMul_4', defined at:
  File "/home/kang/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/kang/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/kang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
 

In [ ]:
pandas.DataFrame((error_scores2.mean(axis=0))).iloc[:,0]